In [102]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# 0. Initial Data Loading and Exploration

## HDI

In [103]:
hdi_df = pd.read_csv('data/HDR21-22_Composite_indices_complete_time_series.csv')
hdi_df.sample(3)

,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
31,CHL,Chile,Very High,LAC,42.0,0.706,0.715,0.723,0.716,0.722,...,11.58,11.66,10.68,11.41,12.57,12.65,13.32,13.50,13.50,13.50
34,CMR,Cameroon,Medium,SSA,151.0,0.452,0.449,0.446,0.439,0.436,...,1.63,1.73,1.82,1.91,1.79,1.73,2.06,2.13,2.13,2.13
92,KIR,Kiribati,Medium,EAP,136.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# Exploring the columns
hdi_df.columns.tolist()

['iso3',
 'country',
 'hdicode',
 'region',
 'hdi_rank_2021',
 'hdi_1990',
 'hdi_1991',
 'hdi_1992',
 'hdi_1993',
 'hdi_1994',
 'hdi_1995',
 'hdi_1996',
 'hdi_1997',
 'hdi_1998',
 'hdi_1999',
 'hdi_2000',
 'hdi_2001',
 'hdi_2002',
 'hdi_2003',
 'hdi_2004',
 'hdi_2005',
 'hdi_2006',
 'hdi_2007',
 'hdi_2008',
 'hdi_2009',
 'hdi_2010',
 'hdi_2011',
 'hdi_2012',
 'hdi_2013',
 'hdi_2014',
 'hdi_2015',
 'hdi_2016',
 'hdi_2017',
 'hdi_2018',
 'hdi_2019',
 'hdi_2020',
 'hdi_2021',
 'le_1990',
 'le_1991',
 'le_1992',
 'le_1993',
 'le_1994',
 'le_1995',
 'le_1996',
 'le_1997',
 'le_1998',
 'le_1999',
 'le_2000',
 'le_2001',
 'le_2002',
 'le_2003',
 'le_2004',
 'le_2005',
 'le_2006',
 'le_2007',
 'le_2008',
 'le_2009',
 'le_2010',
 'le_2011',
 'le_2012',
 'le_2013',
 'le_2014',
 'le_2015',
 'le_2016',
 'le_2017',
 'le_2018',
 'le_2019',
 'le_2020',
 'le_2021',
 'eys_1990',
 'eys_1991',
 'eys_1992',
 'eys_1993',
 'eys_1994',
 'eys_1995',
 'eys_1996',
 'eys_1997',
 'eys_1998',
 'eys_1999',
 'eys_20

In [105]:
# Selecting the columns of interest
hdi_df = hdi_df[['country', 'hdicode', 'hdi_2021', 'region','hdi_rank_2021']]
hdi_df.head()

,country,hdicode,hdi_2021,region,hdi_rank_2021
0,Afghanistan,Low,0.478,SA,180.0
1,Angola,Medium,0.586,SSA,148.0
2,Albania,High,0.796,ECA,67.0
3,Andorra,Very High,0.858,NaN,40.0
4,United Arab Emirates,Very High,0.911,AS,26.0


In [106]:
# Renaming the columns so that they are easier to understand 
hdi_df.rename(columns={'hdi_2021': 'HDI',
                       'country': 'Country',
                       'hdicode': 'HDI Group',
                       'region': 'Region',
                       'hdi_rank_2021': 'HDI_Rank'}, inplace=True)    

In [107]:
hdi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    206 non-null    object 
 1   HDI Group  191 non-null    object 
 2   HDI        202 non-null    float64
 3   Region     151 non-null    object 
 4   HDI_Rank   191 non-null    float64
dtypes: float64(2), object(3)
memory usage: 8.2+ KB


#### Lets look at the null values

In [108]:
# Looking at missing HDI values
hdi_df[hdi_df['HDI'].isnull()]

,Country,HDI Group,HDI,Region,HDI_Rank
108,Monaco,NaN,NaN,NaN,NaN
132,Nauru,NaN,NaN,EAP,NaN
142,Korea (Democratic People's Rep. of),NaN,NaN,EAP,NaN
158,Somalia,NaN,NaN,AS,NaN


In [109]:
# Lookinz at the missing HDI Group values
hdi_df[hdi_df['HDI Group'].isnull()]

,Country,HDI Group,HDI,Region,HDI_Rank
108,Monaco,NaN,NaN,NaN,NaN
132,Nauru,NaN,NaN,EAP,NaN
142,Korea (Democratic People's Rep. of),NaN,NaN,EAP,NaN
158,Somalia,NaN,NaN,AS,NaN
195,Very high human development,NaN,0.896,NaN,NaN
196,High human development,NaN,0.754,NaN,NaN
197,Medium human development,NaN,0.636,NaN,NaN
198,Low human development,NaN,0.518,NaN,NaN
199,Arab States,NaN,0.708,NaN,NaN
200,East Asia and the Pacific,NaN,0.749,NaN,NaN


In [110]:
# Getting the regions (abbreviations)
hdi_df['Region'].value_counts()  

Region
SSA    46
LAC    33
EAP    26
AS     20
ECA    17
SA      9
Name: count, dtype: int64

As we can see we have 4 countries with missing HDI scores. These will have to be dropped as there would be no way to fill in the data for these cells (one could try to look for other datasets). We can also observe that there are summaries of the HDI scores by region. We save these in a separate dataframe so that we can have the data on the countries in one dataframe and the data about the regions in a separate one.

In [111]:
hdi_df_regions = hdi_df.tail(11)
hdi_df_regions

,Country,HDI Group,HDI,Region,HDI_Rank
195,Very high human development,NaN,0.896,NaN,NaN
196,High human development,NaN,0.754,NaN,NaN
197,Medium human development,NaN,0.636,NaN,NaN
198,Low human development,NaN,0.518,NaN,NaN
199,Arab States,NaN,0.708,NaN,NaN
200,East Asia and the Pacific,NaN,0.749,NaN,NaN
201,Europe and Central Asia,NaN,0.796,NaN,NaN
202,Latin America and the Caribbean,NaN,0.754,NaN,NaN
203,South Asia,NaN,0.632,NaN,NaN
204,Sub-Saharan Africa,NaN,0.547,NaN,NaN


In [112]:
hdi_df = hdi_df.drop(hdi_df.tail(11).index)
hdi_df.tail()

,Country,HDI Group,HDI,Region,HDI_Rank
190,Samoa,High,0.707,EAP,111.0
191,Yemen,Low,0.455,AS,183.0
192,South Africa,High,0.713,SSA,109.0
193,Zambia,Medium,0.565,SSA,154.0
194,Zimbabwe,Medium,0.593,SSA,146.0


In [113]:
# Lets drop the countries with NaN HDI values
hdi_df['HDI'].dropna(inplace=True)
hdi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    195 non-null    object 
 1   HDI Group  191 non-null    object 
 2   HDI        191 non-null    float64
 3   Region     151 non-null    object 
 4   HDI_Rank   191 non-null    float64
dtypes: float64(2), object(3)
memory usage: 7.7+ KB


In [114]:
# Lets set the countries as the index
hdi_df.set_index('Country', inplace=True)

We won't drop the rows with null values in the 'Region' row as we can still use the HDI-value of these countries for non-regional analysis.

## IQ

In [115]:
iq_df = pd.read_csv('data/National_IQ.csv')
iq_df.sample(3)

,Rank,Country,Measured IQ,IQ data quality,SchAch,SA direct,SA scaled,SA data quality,Final IQ,Final IQ.1
102,102.0,Paraguay,84.0,6.0,NaN,NaN,NaN,NaN,84,84.0
50,52.0,(Serbia &) Montenegro,93.0,4.0,459.6,90.2,91.0,10.0,92,92.0
19,19.5,New Zealand,99.0,1.0,523.7,100.1,98.7,14.0,98.9,98.9


In [116]:
iq_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             162 non-null    float64
 1   Country          204 non-null    object 
 2   Measured IQ      136 non-null    float64
 3   IQ data quality  137 non-null    float64
 4    SchAch          113 non-null    float64
 5   SA direct        111 non-null    float64
 6   SA scaled        112 non-null    float64
 7   SA data quality  112 non-null    float64
 8    Final IQ        204 non-null    object 
 9    Final IQ.1      204 non-null    float64
dtypes: float64(8), object(2)
memory usage: 16.1+ KB


The values which we are intersted in are ones which are easily interpretable, and relevant to the research question, which aim it is to compare the HDI scores with the IQ scores. To do this, the IQ, the Rank of the country and the country itself ought to be included. 

In [117]:
iq_df.columns

Index(['Rank', 'Country', 'Measured IQ', 'IQ data quality', ' SchAch',
       'SA direct', 'SA scaled', 'SA data quality', ' Final IQ',
       ' Final IQ.1'],
      dtype='object')

In [118]:
columns_of_interest = ['Rank','Country', ' Final IQ']
iq_df = iq_df[columns_of_interest]


In [119]:
iq_df = iq_df.rename(columns=
{' Final IQ': 'IQ',
'Rank': 'IQ_Rank'
})

Since we are searching for potential biases and other correlations between the variables it is crucial that all the IQ values used will be the actual reported ones. Because of this, along with the same methodlogy being applied for the HDI dataset, we have to drop the NaN values rows of the IQ column. 

In [120]:
iq_df['IQ'].dropna(inplace=True)

In [121]:
iq_df.set_index('Country',inplace=True)

In [122]:
iq_df

,IQ_Rank,IQ
Country,,
Singapore,1.0,107.1
China,2.0,105.8
Hong Kong,3.0,105.7
Korea: South,4.5,104.6
Taiwan,4.5,104.6
...,...,...
GuineaBissau,NaN,(69)
Liberia,NaN,(68)
Haiti,NaN,(67)


# 1. Cleaning the Data


# 1. Merge Data


In [123]:
outer_join_df = hdi_df.merge(iq_df, how='outer',on='Country')
outer_join_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, Afghanistan to Sao Tome & Principe
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   HDI Group  191 non-null    object 
 1   HDI        191 non-null    float64
 2   Region     151 non-null    object 
 3   HDI_Rank   191 non-null    float64
 4   IQ_Rank    162 non-null    float64
 5   IQ         204 non-null    object 
dtypes: float64(3), object(3)
memory usage: 13.6+ KB


In [ ]:
missing_in_hdi = outer_join_df[outer_join_df['HDI'].isnull()].index
missing_in_hdi = sorted(missing_in_hdi.tolist())
print("In IQ but missing in HDI: \n", missing_in_hdi)

missing_in_IQ = outer_join_df[outer_join_df['IQ'].isnull()].index
missing_in_IQ = sorted(missing_in_IQ.tolist())
print('In HDI but missing in IQ: \n', missing_in_IQ)
#A few countries on the list seem to have data for both data sets, but are named differently, which makes for the large amount of values (252)
iq_renaming = {
'(Serbia &) Montenegro' : 'Montenegro',
'Central African Rep.' : 'Central African Republic',
'CostaRica' : 'Costa Rica',
'GuineaBissau' : 'Guinea-Bissau',
'Myanmar/Burma' : 'Myanmar',
'Papua N.G.' : 'Papua New Guinea',
'CzechRep.' : 'Czechia',
'Macedonia' : 'North Macedonia'
}
hdi_renaming = {
    'Bolivia (Plurinational State of)' : 'Bolivia',
    'Bosnia and Herzegovina': 'Bosnia',
    'Brunei Darussalam' : 'Brunei',
    'Cabo Verde': 'Cape Verde',
    'Congo' : 'Congo (Brazzaville)',
    'Congo (Democratic Republic of the)' : 'Congo (Zaire)',
    'Hong Kong, China (SAR)' : 'Hong Kong',
    'Iran (Islamic Republic of)' : 'Iran', 
    "Lao People's Democratic Republic" : 'Laos',
    'Moldova (Republic of)' : 'Moldova',
    'Palestine, State of' : 'Palestine',
    'Russian Federation': 'Russia',
    "Côte d'Ivoire": "Cote d'Ivoire",

}
iq_df = iq_df.rename(index = iq_renaming)
hdi_df = hdi_df.rename(index = hdi_renaming)


In [146]:

outer_join_df = hdi_df.merge(iq_df, how='outer',on='Country')
missing_in_hdi = outer_join_df[outer_join_df['HDI'].isnull()].index
missing_in_hdi = sorted(missing_in_hdi.tolist())
print("In IQ but missing in HDI: \n", missing_in_hdi)

missing_in_IQ = outer_join_df[outer_join_df['IQ'].isnull()].index
missing_in_IQ = sorted(missing_in_IQ.tolist())
print('In HDI but missing in IQ: \n', missing_in_IQ)

In IQ but missing in HDI: 
 ['Antigua/Barbuda', 'Bermuda', 'Cook Islands', 'EastTimor', 'England', 'Greenland', "Korea (Democratic People's Rep. of)", 'Korea: North', 'Korea: South', 'Macao', 'Macedonia', 'Mariana Islands', 'Micronesia', 'Monaco', 'Nauru', 'Netherlands Antilles', 'New Caledonia', 'Puerto Rico', 'Samoa (Western)', 'Sao Tome & Principe', 'Scotland', 'Serbia & (Montenegro)', 'Somalia', 'St  Helena', 'St Kitts & Nevis', 'St Lucia', 'St Vincent', 'Swaziland', 'Syria', 'Taiwan', 'Tanzania', 'Tibet', 'Trinidad & Tobago', 'USA', 'Venezuela', 'Vietnam', 'Zanzibar']
In HDI but missing in IQ: 
 ['Antigua and Barbuda', 'Eswatini (Kingdom of)', "Korea (Democratic People's Rep. of)", 'Korea (Republic of)', 'Micronesia (Federated States of)', 'Monaco', 'Nauru', 'North Macedonia', 'Palau', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Serbia', 'South Sudan', 'Syrian Arab Republic', 'Tanzania (United Republi

# 2. INTRODUCTION

In the introduction, provide the description of the problem addressed (the context of your data) and the project objectives.
Very briefly describe the analysis design and how it accomplishes the stated objectives. 
State your research hypotheses in a human-understandable language.
What  can the results be used for?

# 3. DATA CLEANING AND PREPARATION

What did you need to do to clean and prepare your dataset?
Missing values, duplicates, inconsistent data types…


# 4.  DESCRIPTIVE STATISTICS

## 4.1  Univariate analysis
Histogram and metrics introduced in class. Outliers identification. Interpret and discuss your results.

## 4.2  Bivariate analysis
Scatter plots and correlation for pairs of variables of interest. Interpret and discuss your results.



# 5.  DISCUSSION AND PRELIMINARY CONCLUSIONS 

Discuss the initial insights and how they align with the objectives set in the Introduction. Briefly address any limitations or challenges encountered in the data or analysis. Reflect on the implications of these findings and how they might guide future research directions or applications
